In [1]:
import pandas as pd

In [4]:
#test

ratings = pd.read_csv('https://projects.fivethirtyeight.com/soccer-api/club/spi_global_rankings.csv')
ratings = ratings[ratings.league=='Barclays Premier League']

In [5]:
ratings

,rank,prev_rank,name,league,off,def,spi
0,1,1,Manchester City,Barclays Premier League,3.01,0.30,92.98
3,4,4,Liverpool,Barclays Premier League,2.80,0.47,89.02
7,8,8,Tottenham Hotspur,Barclays Premier League,2.35,0.62,81.70
8,9,9,Arsenal,Barclays Premier League,2.29,0.59,81.54
9,10,10,Chelsea,Barclays Premier League,2.19,0.54,81.29
22,23,23,Brighton and Hove Albion,Barclays Premier League,2.00,0.64,76.31
24,25,25,Manchester United,Barclays Premier League,2.10,0.72,75.94
31,32,32,Newcastle,Barclays Premier League,1.94,0.74,72.93
39,40,40,West Ham United,Barclays Premier League,1.83,0.73,71.11
41,42,42,Aston Villa,Barclays Premier League,1.84,0.74,70.97
